In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Wine.csv')
df.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [4]:
df['Customer_Segment'] = df['Customer_Segment'] - 1
print(df['Customer_Segment'].unique())

[0 1 2]


In [5]:
# X - ознаки
X = df[['Alcohol', 'Malic_Acid', 'Ash', 'Ash_Alcanity', 'Magnesium', 'Total_Phenols', 'Flavanoids',
        'Nonflavanoid_Phenols', 'Proanthocyanins', 'Color_Intensity', 'Hue', 'OD280', 'Proline']]

# y - цільова змінна
y = df['Customer_Segment']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [11]:
# Створіть одиночні моделі
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
model_ada = AdaBoostClassifier(n_estimators=100, random_state=42)
model_svc = SVC(probability=True, random_state=42)

# Тренування одиночних моделей
model_rf.fit(X_train, y_train)
model_gb.fit(X_train, y_train)
model_ada.fit(X_train, y_train)
model_svc.fit(X_train, y_train)

# Оцінка точності одиночних моделей на тестовому наборі
y_pred_rf = model_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Точність RandomForestClassifier: {accuracy_rf}")

# Оцінка точності одиночних моделей на тестовому наборі
y_pred_gb = model_gb.predict(X_test)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Точність GradientBoostingClassifier: {accuracy_gb}")

# Оцінка точності одиночних моделей на тестовому наборі
y_pred_ada = model_ada.predict(X_test)
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print(f"Точність AdaBoostClassifier: {accuracy_ada}")

# Оцінка точності одиночних моделей на тестовому наборі
y_pred_svc = model_svc.predict(X_test)
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print(f"Точність SVC: {accuracy_svc}")

# Створіть паралельний гетерогенний ансамбль
ensemble_parallel = VotingClassifier(estimators=[
    ('rf', model_rf),
    ('gb', model_gb),
    ('ada', model_ada),
    ('svc', model_svc)
], voting='hard')

# Тренуйте ансамбль
ensemble_parallel.fit(X_train, y_train)

# Оцінка точності на тестовому наборі
y_pred_ensemble_parallel = ensemble_parallel.predict(X_test)
accuracy_ensemble_parallel = accuracy_score(y_test, y_pred_ensemble_parallel)

print(f"Точність паралельного гетерогенного ансамблю: {accuracy_ensemble_parallel}")

Точність RandomForestClassifier: 0.9722222222222222
Точність GradientBoostingClassifier: 0.9444444444444444
Точність AdaBoostClassifier: 0.8333333333333334
Точність SVC: 0.7777777777777778
Точність паралельного гетерогенного ансамблю: 0.9722222222222222


In [16]:
from sklearn.model_selection import GridSearchCV

# Створення паралельного гетерогенного ансамблю з можливістю підбору параметрів
ensemble_parallel = VotingClassifier(estimators=[
    ('rf', model_rf),
    ('gb', model_gb),
    ('ada', model_ada),
    ('svc', model_svc)
], voting='hard')

# Задайте спрощену сітку параметрів для пошуку
param_grid = {
    'rf__n_estimators': [50, 100],
    'gb__n_estimators': [50, 100],
    'ada__n_estimators': [50, 100],
    'svc__C': [0.1, 1, 5],
    'svc__kernel': ['linear', 'rbf']
}
# Створіть об'єкт GridSearchCV
grid_search = GridSearchCV(ensemble_parallel, param_grid, cv=5)

# Проведіть пошук оптимальних параметрів
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=VotingClassifier(estimators=[('rf',
                                                     RandomForestClassifier(random_state=42)),
                                                    ('gb',
                                                     GradientBoostingClassifier(random_state=42)),
                                                    ('ada',
                                                     AdaBoostClassifier(n_estimators=100,
                                                                        random_state=42)),
                                                    ('svc',
                                                     SVC(probability=True,
                                                         random_state=42))]),
             param_grid={'ada__n_estimators': [50, 100],
                         'gb__n_estimators': [50, 100],
                         'rf__n_estimators': [50, 100], 'svc__C': [0.1, 1, 5],
                         'svc__kernel': ['linear', 'rbf']})

In [17]:
# Отримайте найкращі параметри
best_params = grid_search.best_params_
print(f"Оптимальні параметри для ансамблю: {best_params}")

# Оцініть точність ансамблю з найкращими параметрами
y_pred_best_params = grid_search.predict(X_test)
accuracy_best_params = accuracy_score(y_test, y_pred_best_params)
print(f"Точність ансамблю з найкращими параметрами: {accuracy_best_params}")

Оптимальні параметри для ансамблю: {'ada__n_estimators': 50, 'gb__n_estimators': 50, 'rf__n_estimators': 50, 'svc__C': 5, 'svc__kernel': 'rbf'}
Точність ансамблю з найкращими параметрами: 0.9722222222222222


In [19]:
# Отримання важливості предикторів для Random Forest
rf_feature_importances = model_rf.feature_importances_

# Отримання важливості предикторів для Gradient Boosting
gb_feature_importances = model_gb.feature_importances_

# Отримання важливості предикторів для AdaBoost
ada_feature_importances = model_ada.feature_importances_

# Виведення результатів
print("Важливість предикторів для Random Forest:")
print(rf_feature_importances)

print("\nВажливість предикторів для Gradient Boosting:")
print(gb_feature_importances)

print("\nВажливість предикторів для AdaBoost:")
print(ada_feature_importances)


Важливість предикторів для Random Forest:
[0.12903568 0.02163008 0.0149994  0.02122211 0.03163118 0.03606108
 0.19987868 0.01631    0.02467837 0.19167399 0.07604694 0.09506015
 0.14177233]

Важливість предикторів для Gradient Boosting:
[2.40030252e-03 1.69850510e-02 2.30571126e-02 9.70298821e-03
 2.19538946e-02 2.74309360e-03 9.87422159e-02 3.01306414e-04
 5.52984062e-05 3.13715714e-01 4.42433633e-03 2.55315725e-01
 2.50602961e-01]

Важливість предикторів для AdaBoost:
[0.   0.   0.   0.   0.   0.   0.49 0.   0.   0.01 0.49 0.01 0.  ]


<h4>Random Forest: </h4>

    Значення важливості розподілені між всіма предикторами.
    Найважливішими є 'Flavanoids' (0.1999) і 'Color_Intensity' (0.1917).
    Предиктори, які мають менше значення важливості, внесли менший вклад у прийняття рішення.
<h4>Gradient Boosting:</h4>

    'Color_Intensity' (0.3137), 'OD280' (0.2553), та 'Proline' (0.2506) є найважливішими предикторами.
     Деякі предиктори, такі як 'Total_Phenols' і 'Hue', також внесли свій вклад, але менший порівняно з топ-3.
<h4>AdaBoost:</h4>

    Тільки два предиктори, 'Flavanoids' (0.49) і 'Color_Intensity' (0.49), мають значення важливості відмінні від нуля.
    Інші предиктори, очевидно, не враховуються моделлю.

In [21]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
# Створення моделей для стекінгу
estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
    ('ada', AdaBoostClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(probability=True, random_state=42))
]

# Створення стекінгу
ensemble_sequential = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# Тренуйте ансамбль
ensemble_sequential.fit(X_train, y_train)

# Оцінка точності на тестовому наборі
y_pred_ensemble_sequential = ensemble_sequential.predict(X_test)
accuracy_ensemble_sequential = accuracy_score(y_test, y_pred_ensemble_sequential)

print(f"Точність послідовного гетерогенного ансамблю: {accuracy_ensemble_sequential}")


Точність послідовного гетерогенного ансамблю: 0.9722222222222222
